In [3]:
import numpy as np
from Meshing.Meshing import *
from Fields.Fields import *
from Kernels.Kernels import *
from Solvers.Solvers import *
from Subchannel.FluidRelation import FluidRelation
from Subchannel.Channel import Channel
from Subchannel.Channel import ChannelInterface



In [4]:
gravity = 9.81
Dh = 0.0158276279311347000000
perim = 0.07267978843558340
area = 0.000287587162368000000
temp_tolerance = 1e-10
max_temp_iterations = 1000
nZones=25
L0 = 0.0
L1 = 1.74
fluid = FluidRelation(cp=1983, mu=0.00744, k=1.44, rho_0 = 2715.13, drho_dT=-0.513)
pressure_bc = 0.0
T_bc = 900.0
mdot_bc = 0.15
fric = 'type1'
_dt = 1e321
heat_source = [0.0]*nZones # heat source in W/m3
#################
# CHANNEL SETUPS
#################
ch = Channel(gravity=gravity,
             Dh=Dh,
             area=area,
             temp_tolerance=temp_tolerance,
             max_temp_iterations=max_temp_iterations,
             nZones=nZones,
             L0=L0,
             L1=L1,
             fluid=fluid,
             pressure_bc=pressure_bc,
             T_bc=T_bc,
             mdot_bc=mdot_bc,
             fric=fric,
             heat_source=heat_source)

upper_plenum = Channel(gravity=gravity,
                       Dh=Dh,
                       area=area,
                       temp_tolerance=temp_tolerance,
                       max_temp_iterations=max_temp_iterations,
                       nZones=nZones,
                       L0=0,
                       L1=0.15,
                       fluid=fluid,
                       pressure_bc=pressure_bc,
                       T_bc=1.0,
                       mdot_bc=1.0,
                       fric='none',
                       heat_source=0.0)

external_loop = Channel(gravity=gravity,
                       Dh=Dh,
                       area=area*100,
                       temp_tolerance=temp_tolerance,
                       max_temp_iterations=max_temp_iterations,
                       nZones=nZones,
                       L0=0,
                       L1=5.0,
                       fluid=fluid,
                       pressure_bc=pressure_bc,
                       T_bc=1.0,
                       mdot_bc=1.0,
                       fric='none',
                       heat_source=0.0)


# Interfaces to handle data passing from one channel to another.
ch_to_plenum = ChannelInterface(ch1=ch, ch2=upper_plenum)
plenum_to_ex = ChannelInterface(ch1=upper_plenum, ch2=external_loop)
ex_to_ch = ChannelInterface(ch1=external_loop, ch2=ch)

# Solution order - main channel -> plenum -> external loop -> channel
ch.solve_channel_TH(_dt=_dt)
ch_to_plenum.update_interface_conditions()

upper_plenum.solve_channel_TH(_dt=_dt)
plenum_to_ex.update_interface_conditions()

external_loop.solve_channel_TH(_dt=_dt)
ex_to_ch.update_interface_conditions()



# End



Channel solved after 1 iterations!
Channel solved after 2 iterations!
Channel solved after 2 iterations!


In [ ]:
tracer_names = ['c1', 'c2', 'c3', 'c4', 'c5', 'c6']
initial_value_tracers = 0.0
scheme_tracers = 'upwind'
decay_consts = [0.05, 0.10, 0.15, 0.20, 0.25, 0.30]
boundary = 'lower' # must be lower anyways
phi_tracer = 1.0
rho_tracer = 1.0
fsrc = 0.0
#### NEW STUFFs

ch.add_tracer_to_channel(name=tracer_names[0], initial_value=initial_value_tracers,
                         scheme='upwind', decay_const=decay_consts[0],
                         boundary=boundary, phi=phi_tracer, rho=rho_tracer, source=fsrc)
ch.solve_all_tracers(_dt=1e321)

AttributeError: 'list' object has no attribute 'phi'

In [ ]:
### ADDED THESE TO TO CHANNEL CODE - NOW NEED TO INCORPORATE PASSING OF OUTGOING VALUES INTO CHANNEL.PY
### IN GET_OUTLET_CONDITIONS() - AND - UPDATE_INTERFACE_CONDITIONS() SO THAT TRACERS ARE HANDLED SEAMLESSLY.
# DNP EQUATION
c1 = ScalarField(name='c1', initial_value=0.0, mesh=ch.mesh)

# KERNELS
c1_adv = AdvectionKernel(field=c1, mesh=ch.mesh, w=ch.velocity_faces, scheme='quick', rho=1.0)
c1_decay = ImplicitReactionKernel(field=c1, mesh=ch.mesh,lam=0.5)

# BCS
bc = AdvectedInletFluxBC(field=c1, mesh=ch.mesh, boundary='lower', phi=1.0, w=ch.velocity_faces, rho=1.0)

# SOLVER
solver = BasicSolver(kernels=[c1_decay, c1_adv], bcs=[bc], field=c1)
solver.solve()

print(c1.T)


In [ ]:
ch.velocity_faces.T

In [ ]:
ch.temp.T

In [ ]:
# velocities at faces and at centroids
plt.plot(ch.mesh.nodeCoords, ch.velocity_faces.T, 'k--s')
plt.plot(ch.mesh.centroids, ch.mdot.T / ch.rho.T / ch.area, 'r--s')

In [ ]:
external_loop.temp.T

In [ ]:
ch.T_bc

In [ ]:
ch.b_energy

In [ ]:
ch.temp.T

In [ ]:
np.pi * Dh**2 / 4

In [ ]:
0.18*area*2200

In [ ]:
ch.heat_source

In [ ]:
ch.h.T

In [ ]:
ch.h_bc